# Current Testing Battery

In [1]:
import sys
import statistics
import numpy
from itertools import product
from timeit import default_timer as timer
from tqdm import tqdm
sys.path.append("..")
import gillespy2
from scipy import stats

In [2]:
from gillespy2.basic_ssa_solver import BasicSSASolver
from gillespy2.ssa_c_solver import SSACSolver
from gillespy2.basic_root_solver import BasicRootSolver
from gillespy2.optimized_ssa_solver import OptimizedSSASolver
from gillespy2.basic_tau_leaping_solver import BasicTauLeapingSolver
from gillespy2.basic_tau_hybrid_solver import BasicTauHybridSolver
from gillespy2.example_models import *

In [3]:
modelList = [Example(), Trichloroethylene(), MichaelisMenten(), Schlogl()]

solverList = [BasicSSASolver, SSACSolver, BasicRootSolver, OptimizedSSASolver, 
              OptimizedSSASolver(), BasicTauLeapingSolver, BasicTauLeapingSolver(), 
             BasicTauHybridSolver, BasicTauHybridSolver()]
timingList = []

precompile_solver = True


for model, solver in product(modelList, solverList):
    medianList = []
    exterior_stats = []
    if precompile_solver and (solver == SSACSolver or isinstance(solver, SSACSolver)): #precompile SSACSolver to save time in repated tests
        solver = SSACSolver(model)
    for i in tqdm(range(30), desc = 'Model: {0}, Solver: {1}'.format(model.name, solver)):
        standard_results = model.run()
        start = timer()
        test_results = model.run(solver=solver)
        stop = timer()
        medianList.append(stop-start)
        interior_stats = []
        for species in standard_results[0].keys():
            if isinstance(test_results, dict):
                interior_stats.append(stats.ks_2samp(standard_results[0][species], test_results[species])[0])
            else:
                interior_stats.append(stats.ks_2samp(standard_results[0][species], test_results[0][species])[0])
                
        exterior_stats.append(statistics.median(interior_stats))
    median = statistics.median(medianList)
    timingList.append([model, solver, median])

Model: Example, Solver: <class 'gillespy2.basic_ssa_solver.BasicSSASolver'>: 100%|██████████| 30/30 [00:09<00:00,  3.24it/s]
Model: Example, Solver: <gillespy2.ssa_c_solver.SSACSolver object at 0x7f0ce8852f60>: 100%|██████████| 30/30 [00:14<00:00,  1.91it/s]
Model: Example, Solver: <class 'gillespy2.basic_root_solver.BasicRootSolver'>: 100%|██████████| 30/30 [00:13<00:00,  1.98it/s]
Model: Example, Solver: <class 'gillespy2.optimized_ssa_solver.OptimizedSSASolver'>: 100%|██████████| 30/30 [00:09<00:00,  3.52it/s]
Model: Example, Solver: <gillespy2.optimized_ssa_solver.OptimizedSSASolver object at 0x7f0ce88b9ac8>: 100%|██████████| 30/30 [00:08<00:00,  3.33it/s]
Model: Example, Solver: <class 'gillespy2.basic_tau_leaping_solver.BasicTauLeapingSolver'>: 100%|██████████| 30/30 [00:09<00:00,  3.06it/s]
Model: Example, Solver: <gillespy2.basic_tau_leaping_solver.BasicTauLeapingSolver object at 0x7f0ce88bb7f0>: 100%|██████████| 30/30 [00:09<00:00,  2.60it/s]
Model: Example, Solver: <class 'gi

In [4]:
interior_stats

[0.0, 0.0, 0.0, 0.0, 0.010101010101010166]